In [1]:
!pip install tensorflow==2.15

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation

In [1]:
!git clone -b roshan_b https://github.com/roshant2003/Railway-Track-Fault-Detection-Project
!cp -r Railway-Track-Fault-Detection-Project/Railway/ /content/Dataset
!cp -r Railway-Track-Fault-Detection-Project/Railway_pre/ /content/Dataset_Pre

Cloning into 'Railway-Track-Fault-Detection-Project'...
remote: Enumerating objects: 10334, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10334 (delta 1), reused 11 (delta 1), pack-reused 10322 (from 1)
Receiving objects: 100% (10334/10334), 599.19 MiB | 22.76 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (10309/10309), done.


In [22]:
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

data_dir = 'Dataset_Pre'

n_classes = 7

# Image parameters
image_size = 224

# Training parameters
loss_metrics = ['accuracy']
n_epochs = 100
early_stop_patience = 10

steps_per_epoch_training = 10
steps_per_epoch_validation = 10

batch_size_training = 20
batch_size_validation = 20

# Define ImageDataGenerator with augmentation
datagen = ImageDataGenerator(
        zoom_range=0.15,
        shear_range=0.15,
        horizontal_flip=True,
        validation_split=0.2,
        rescale=1.0/255)

# Training and validation generators
train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size_training,
        class_mode='categorical',
        subset='training')

validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size_validation,
        class_mode='categorical',
        subset='validation')

# Load the InceptionV3 model without the top layers (for transfer learning)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Add new layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling

# Add fully connected layer with BatchNormalization and Dropout
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)  # Batch Normalization
x = Dropout(0.5)(x)  # Dropout for regularization

# Add another Dense layer for the classification with softmax activation
predictions = Dense(n_classes, activation='softmax')(x)

# Define the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Fine-tune: Unfreeze the top layers of the InceptionV3 model
for layer in base_model.layers[:249]:
    layer.trainable = False  # Freeze layers
for layer in base_model.layers[249:]:
    layer.trainable = True   # Unfreeze the last layers for fine-tuning

# Compile the model with a lower learning rate for fine-tuning
optimizer = Adam(learning_rate=1e-4)  # Fine-tuning usually requires a lower learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=loss_metrics)




Found 4125 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


In [23]:

from tensorflow.keras import optimizers

# Compile model
sgd = optimizers.SGD(learning_rate=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss=objective_function, metrics=loss_metrics)



In [24]:
import os
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

# Ensure that the working directory exists
if not os.path.exists('../working/'):
    os.makedirs('../working/')

# Prepare early stopping to avoid overfitting
cb_early_stopper = EarlyStopping(monitor='val_loss', patience=early_stop_patience)
cb_checkpointer = ModelCheckpoint(filepath='../working/best', monitor='val_loss', save_best_only=True, mode='auto')

# Fit the model
fit_history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch_training,
        epochs=n_epochs,
        validation_data=validation_generator,
        validation_steps=steps_per_epoch_validation,
        callbacks=[cb_checkpointer, cb_early_stopper]
)

# Ensure the model checkpoint file exists before loading
if os.path.exists("../working/best"):
    model.load_weights("../working/best")
else:
    print("Checkpoint file not found!")

# Save the model to a valid path
model.save("InceptionV3_RTFD.keras")


Epoch 1/100
10/10 [==============================] - 42s 4s/step - loss: 2.1151 - accuracy: 0.4950 - val_loss: 19.8480 - val_accuracy: 0.3550
Epoch 2/100
10/10 [==============================] - 8s 832ms/step - loss: 1.5744 - accuracy: 0.7300 - val_loss: 69.8477 - val_accuracy: 0.3550
Epoch 3/100
10/10 [==============================] - 8s 821ms/step - loss: 0.6749 - accuracy: 0.8757 - val_loss: 23.9181 - val_accuracy: 0.1450
Epoch 4/100
10/10 [==============================] - 6s 608ms/step - loss: 0.8970 - accuracy: 0.8150 - val_loss: 25.9686 - val_accuracy: 0.1700
Epoch 5/100
10/10 [==============================] - 6s 605ms/step - loss: 0.6181 - accuracy: 0.8750 - val_loss: 62.9572 - val_accuracy: 0.3650
Epoch 6/100
10/10 [==============================] - 32s 4s/step - loss: 0.3423 - accuracy: 0.9250 - val_loss: 13.1972 - val_accuracy: 0.4050
Epoch 7/100
10/10 [==============================] - 6s 677ms/step - loss: 0.5714 - accuracy: 0.8850 - val_loss: 28.0849 - val_accuracy: 0.3

In [25]:
# Load TF Keras model
import tensorflow as tf
model = tf.keras.models.load_model("InceptionV3_RTFD.keras")

# Accuracy test with complete dataset
import os
import cv2
import progressbar
import numpy as np

def classify(image):
    # Pre-processing
    image = cv2.resize(image, (224, 224))
    image = np.array(np.expand_dims(image, 0), dtype=np.float32)
    image = image / 255.
    output = model.predict(image, verbose=0)
    return np.argmax(output, axis=1)[0]

def test_model(folder, classes, length):
    class_idx = 0
    curr_count = 0
    correct_count = 0
    with progressbar.ProgressBar(max_value=length) as bar:
        for c in classes:
            for f in os.listdir(os.path.join(folder, c)):
                bar.update(curr_count)
                image = cv2.imread(os.path.join(folder, c, f))
                out = classify(image)

                if out == class_idx: correct_count += 1
                curr_count += 1
            class_idx += 1

    print(f"Total images           : {curr_count}")
    print(f"Correct predictions    : {correct_count}")
    print(f"Incorrect predictions  : {curr_count - correct_count}")
    print(f"Accuracy               : {correct_count / curr_count * 100} %")

# TF Keras
classes = list(train_generator.class_indices.keys())
test_model('Dataset_Pre', classes, 5153)


 11% (574 of 5153) |##                   | Elapsed Time: 0:00:53 ETA:   0:07:03WARNING:tensorflow:Detecting that an object or model or tf.train.Checkpoint is being deleted with unrestored values. See the following logs for the specific values in question. To silence these warnings, use `status.expect_partial()`. See https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint#restorefor details about the status object returned by the restore function.
100% (5153 of 5153) |####################| Elapsed Time: 0:08:06 Time:  0:08:06


Total images           : 5153
Correct predictions    : 4141
Incorrect predictions  : 1012
Accuracy               : 80.3609547836212 %
